Input data

In [ ]:
# Define paths
input_folder = r"D:\River connectivity\Whole Danube Reach in Hungary\Historical Danube-Reprojected\Graph theory calculations\output-graph model"
excel_output = r"D:\River connectivity\Whole Danube Reach in Hungary\xlsx\graph_metrics_historical danube.xlsx"

Processing code

In [ ]:
import os
import re
import geopandas as gpd
import networkx as nx
import pandas as pd
from shapely.geometry import Point

# Regular expression pattern to extract the date from the filename
date_pattern = r"\d{4}-\d{2}-\d{2}"

# Initialize list to store results
results = []

# Function to create nodes at intersections
def create_intersection_nodes(edges_gdf):
    nodes = set()
    for i, line1 in enumerate(edges_gdf.geometry):
        for j, line2 in enumerate(edges_gdf.geometry):
            if i >= j:
                continue
            intersection = line1.intersection(line2)
            if intersection.is_empty:
                continue
            if intersection.geom_type == 'Point':
                nodes.add(intersection)
            elif intersection.geom_type == 'MultiPoint':
                for point in intersection.geoms:
                    nodes.add(point)
    # Convert nodes set to GeoDataFrame
    return gpd.GeoDataFrame(geometry=list(nodes), crs=edges_gdf.crs)

# Process each shapefile in the folder
for filename in os.listdir(input_folder):
    if filename.endswith(".shp"):
        # Extract the date from the filename
        date_match = re.search(date_pattern, filename)
        if not date_match:
            continue  # Skip files without a date
        date_str = date_match.group(0)

        # Load the shapefile
        edges_gdf = gpd.read_file(os.path.join(input_folder, filename))
        edges_gdf = edges_gdf.to_crs(epsg=32634)  # Ensure CRS is projected for distance calculations

        # Create intersection nodes
        nodes_gdf = create_intersection_nodes(edges_gdf)
        
        # Create a graph
        G = nx.Graph()
        for idx, row in nodes_gdf.iterrows():
            G.add_node(idx, pos=(row.geometry.x, row.geometry.y))
        
        # Add edges to the graph
        for idx, row in edges_gdf.iterrows():
            start_point, end_point = row.geometry.coords[0], row.geometry.coords[-1]
            start_node = nodes_gdf.distance(Point(start_point)).idxmin()
            end_node = nodes_gdf.distance(Point(end_point)).idxmin()
            if start_node != end_node:
                G.add_edge(start_node, end_node)

        # Calculate graph-theoretic metrics
        alpha = nx.edge_connectivity(G)
        avg_shortest_path = nx.average_shortest_path_length(G) if nx.is_connected(G) else None
        diameter = nx.diameter(G) if nx.is_connected(G) else None
        fiedler_value = nx.algebraic_connectivity(G) if nx.is_connected(G) else None
        assortativity = nx.degree_assortativity_coefficient(G)
        density = nx.density(G)

        # Store the results in a dictionary
        results.append({
            "Date": date_str,
            "Alpha": alpha,
            "Average Shortest Path": avg_shortest_path,
            "Diameter": diameter,
            "Fiedler Value": fiedler_value,
            "Assortativity": assortativity,
            "Density": density
        })

# Convert results to DataFrame and export to Excel
df = pd.DataFrame(results)
df.to_excel(excel_output, index=False)
print(f"Graph metrics exported to {excel_output}")
